In [1]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\yujit\.julia\packages\Plots\0MO2T\src\backends.jl:318


Plots.PlotlyBackend()

In [73]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")

Main.R3BP

In [74]:
param3b = SailorMoon.dyanmics_parameters()

Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.01709689063726318, 7.601281331451572)

In [75]:
τ = 0.00001
tmax = 0.4  # N = kg m / s^2
mdot = tmax / 9.8 / 3500  # kg/s -> Isp = 3500s

# # normalize
mstar = 5.972e24 / param3b.mu1 
tmax = tmax / mstar / param3b.lstar * (param3b.tstar)^2 
mdot = mdot / mstar * param3b.tstar 
mf = 3000 / mstar  

n = 1
SCParam = [mdot, tmax, mf]

3-element Vector{Float64}:
 7.247336249409605e-25
 2.4273778713483754e-20
 4.9624049617734895e-22

In [76]:
# set up LPOArr 

lp = 2
Az_km = 28000.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="z")
res.flag

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix

ys0 = R3BP.get_eigenvector(monodromy, true, 1)
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

Halo guess Az_km: 28000.0
Linear stability ν = 290.9383214356218


In [77]:
# extract optimization var
ϕ = 0.5
θm = deg2rad(210)

tof      = (70 * 86400) / param3b.tstar
c_launch = [mf*1.2, 0.1, 0.0, 0.0]  # m0, v∞1, α1, δ1
c_arr    = [ϕ, θm]
tau1     = [0.0001, 0.0, 0.0]  # discretization numbers are the same for the first & second arc
tau2     = [0.0001, 0.0, 0.0]

# make a decision vector x 
x = []
append!(x,tof)
append!(x,c_launch)
append!(x,c_arr)
append!(x,tau1)
append!(x,tau2)
print(x)

Any[16.09793576177804, 5.954885954128187e-22, 0.1, 0.0, 0.0, 0.5, 3.6651914291880923, 0.0001, 0.0, 0.0, 0.0001, 0.0, 0.0]

In [78]:
# set up the default ODE
state0 = zeros(7)
tspan = zeros(2)
params = zeros(10)

Propagator = SailorMoon.ODEPropagator(
    Tsit5(), 1e-12, 1e-12, ODEProblem(SailorMoon.rhs_bcr4bp_with_mass!, state0, tspan, params)
);

In [79]:
# propagate
res, _, _, _, _ = SailorMoon.sf_propagate(x, n, SCParam, Propagator, param3b, LPOArrival)
print(res)

[-10.861673398263642, -2.3872457804659946, 4.241447115472114, 4.258590909625785, 16.928299090193892, -0.7290521780755062, -9.924693256392276e-23]